<a href="https://colab.research.google.com/github/Rajeshchag/NNDL/blob/main/Assignment_2_NNDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
#mount the drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
#import certain packages
import numpy as np
import math as mt
import random
import pandas as pd

In [60]:
#read the data set
data=pd.read_csv('/content/drive/My Drive/Colab Notebooks/dataset2.csv')
data.head()


,Date,Time,Load (kW),Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,01.09.2018,0:00,5551.82208,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1:00,4983.17184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2:00,4888.39680,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,3:00,5072.95872,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,4:00,5196.25980,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
x = []
y = []
for i in range(24, data.shape[0]):
    x.append(data['Load (kW)'].iloc[i-24])
    y.append(data['Load (kW)'].iloc[i])

In [62]:
data = pd.DataFrame({'Previous Day Load' : x, 'Present Day Load' : y})
data.head()

,Previous Day Load,Present Day Load
0,5551.82208,4931.26380
1,4983.17184,4775.53968
2,4888.39680,4713.81120
3,5072.95872,4689.02628
4,5196.25980,4844.75040


In [63]:
#normalisation
maxX = np.max(data['Previous Day Load'])
minX = np.min(data['Previous Day Load'])
maxY = np.max(data['Present Day Load'])
minY = np.min(data['Present Day Load'])

data['Previous Day Load'] = (data['Previous Day Load'] - minX) / (maxX - minX)
data['Present Day Load'] = (data['Present Day Load'] - minY) / (maxY - minY)

In [64]:
#split the data
from sklearn.model_selection import train_test_split
x = data['Previous Day Load']
y = data['Present Day Load']
trainX, testX, trainY, testY = train_test_split(x, y, test_size=0.1, random_state=101)

In [ ]:
m = np.random.uniform(-5, 5)
c = np.random.uniform(-5, 5)
eta = 0.01
epochs = 500
vm = 0
vc = 0
v = 0.8

# Loop for epochs
for _ in range(epochs):
    
    for i in range(trainX.shape[0]):
        # Calculating Grad M
        gradM = -1 * (trainY.iloc[i] - m * trainX.iloc[i] - c) * trainX.iloc[i]
    
        # Calculating Grad C
        gradC = -1 * (trainY.iloc[i] - m * trainX.iloc[i] - c)
        
        # Calculating updated values of vm and vc
        vm = (v * vm) - (eta * gradM)
        vc = (v * vc) - (eta * gradC)
        
        # Updating m and c values
        m += vm
        c += vc
print("The parameter m value is : ",m)
print("The constant value c is : ",c)

The parameter m value is :  0.687293060487023
The constant value c is :  0.12260026923866679


In [65]:

train_pred = [m * trainX.iloc[i] + c for i in range(trainX.shape[0])]
test_pred = [m * testX.iloc[i] + c for i in range(testX.shape[0])]

# Denormalization of Output Values
train_pred = [i * (maxY - minY) + minY for i in train_pred]
test_pred = [i * (maxY - minY) + minY for i in test_pred]
trainY = [i * (maxY - minY) + minY for i in trainY]
testY = [i * (maxY - minY) + minY for i in testY]

# Training Error Calculation
training_MAE = sum([abs(train_pred[i] - trainY[i]) for i in range(len(trainY))]) / len(trainY)
training_MSE = sum([(train_pred[i] - trainY[i]) ** 2 for i in range(len(trainY))]) / len(trainY)
training_RMSE = training_MSE ** 0.5
print('Training Error :')
print(f'MAE : {training_MAE}')
print(f'MSE : {training_MSE}')
print(f'RMSE : {training_RMSE}')

Training Error :
MAE : 550.1949063695762
MSE : 541489.0761420903
RMSE : 735.8594132999117


In [66]:

testing_MAE = sum([abs(test_pred[i] - testY[i]) for i in range(len(testY))]) / len(testY)
testing_MSE = sum([(test_pred[i] - testY[i]) ** 2 for i in range(len(testY))]) / len(testY)
testing_RMSE = testing_MSE ** 0.5
print('Testing Error :')
print(f'MAE : {testing_MAE}')
print(f'MSE : {testing_MSE}')
print(f'RMSE : {testing_RMSE}')


Testing Error :
MAE : 563.0197628835851
MSE : 526241.2746310369
RMSE : 725.424892480977


In [67]:
pd.DataFrame({'Actual' : testY, 'Predicted' : test_pred})

,Actual,Predicted
0,6161.624640,5590.738565
1,5761.792440,6768.369149
2,7390.270800,6286.903370
3,4597.368840,5757.869543
4,5242.712040,5866.504679
...,...,...
211,6764.319072,7331.364847
212,7390.270800,6601.880982
213,6550.077600,6428.321890
214,5722.043040,5491.209925


In [68]:
x = float(input('give load of present hour on previouus day : '))
x = (x - minX) / (maxX - minX)
prediction = m * x + c
prediction = (prediction * (maxY - minY)) + minY
print('Predicted load of present hour :', prediction)

give load of present hour on previouus day : 345
Predicted load of present hour : 1963.272212245656
